In [5]:
import openai
import pandas as pd
global listhehe
listhehe= []


In [6]:
openai.api_key = "YOUR OWN API KEY"
model_engine = "gpt-3.5-turbo"

In [7]:
def get_bulletpoints(link):
    global listhehe
    response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Extract 3 to 5 brief ( up to 8 words ) bulletpoints but with no indexing that can pass as tasks from the following link:" + str(link)},
    ])
    message = response.choices[0]['message']
    prompt = message['content']
    listhehe = message['content'].splitlines()
    return prompt
    

In [8]:
prompt = get_bulletpoints('https://adsai.buas.nl/Study%20Content/DeepLearning/11.%20CNN%20day%201.html')
print(listhehe)

['- Define Convolutional Neural Networks (CNN)', '- Understand the architecture of a CNN', '- Implement a CNN using Keras', '- Train a CNN on a dataset', '- Fine-tune a pre-trained CNN model']


In [9]:
template = ['Date', 'Status', 'Task' ,'Description/Planning','Task Type','Most suited ILO', 'Est Hours',
             'Actual Hours','Diff','Evidence link','Task reflection / notes']
df = pd.DataFrame(columns = template)

In [10]:
df = df.append(pd.DataFrame(listhehe, columns=['Description/Planning']), ignore_index=True)

C:\Users\bogda\AppData\Local\Temp\ipykernel_30560\1861502454.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(listhehe, columns=['Description/Planning']), ignore_index=True)


In [11]:
def get_hours(prompt):
    response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": 'Give me a simple non-indexed list that only contains time values in hours that add up to 8 hours for the following tasks,please do not include the tasks and put every entry in the list on a different line and also do not include the word "hours"' + str(prompt)},
    ])
    message = response.choices[0]['message']
    hourlist = message['content']
    return hourlist

hourlist = get_hours(prompt)
hourlist = hourlist.splitlines()



In [12]:
listhehe_df = pd.DataFrame()
listhehe_df["Description/Planning"] = listhehe
hourlist_df = pd.DataFrame()
hourlist_df['Est Hours'] = hourlist

listhehe_df.reset_index(drop = True, inplace = True)
hourlist_df.reset_index(drop = True, inplace = True)

""" listhehe_df['Task Description/Planning'] = listhehe_df['Task Description/Planning'].str.replace('\n', '')
hourlist_df['Est Hours'] = hourlist_df['Est Hours'].str.replace('\n', '') """


" listhehe_df['Task Description/Planning'] = listhehe_df['Task Description/Planning'].str.replace('\n', '')\nhourlist_df['Est Hours'] = hourlist_df['Est Hours'].str.replace('\n', '') "

In [13]:
yes = pd.concat([df, listhehe_df], axis = 1, sort = False).drop_duplicates()
yes = pd.concat([yes, hourlist_df], axis = 1, sort = False).drop_duplicates()

In [14]:
yes = yes.loc[:,~yes.columns.duplicated()]

In [15]:
yes['Est Hours'] = hourlist_df['Est Hours']
yes['Actual Hours'] = hourlist_df['Est Hours']
yes['Diff'] = 0
yes['Task'] = 'Max 4 hours tasks'

In [16]:
yes['Actual Hours'] = yes['Actual Hours'].astype(float)
yes['Est Hours'] = yes['Est Hours'].astype(float)

In [17]:
yes = yes.reset_index(drop=True)

In [18]:
yes

,Date,Status,Task,Description/Planning,Task Type,Most suited ILO,Est Hours,Actual Hours,Diff,Evidence link,Task reflection / notes
0,NaN,NaN,Max 4 hours tasks,- Define Convolutional Neural Networks (CNN),NaN,NaN,1.0,1.0,0,NaN,NaN
1,NaN,NaN,Max 4 hours tasks,- Understand the architecture of a CNN,NaN,NaN,2.0,2.0,0,NaN,NaN
2,NaN,NaN,Max 4 hours tasks,- Implement a CNN using Keras,NaN,NaN,1.5,1.5,0,NaN,NaN
3,NaN,NaN,Max 4 hours tasks,- Train a CNN on a dataset,NaN,NaN,2.5,2.5,0,NaN,NaN
4,NaN,NaN,Max 4 hours tasks,- Fine-tune a pre-trained CNN model,NaN,NaN,1.0,1.0,0,NaN,NaN


In [20]:
yes.to_excel('C:/Users/bogda/Documents/GitHub/Lazy-Logger/LazyData_Save.xlsx', index= False )